In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb

In [2]:
#Leo el csv de eventos

events_df = pd.read_csv("data/fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv", low_memory=False)

In [3]:
events_df.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Generación de features
---

In [4]:
features_df = pd.DataFrame()

In [5]:
features_df['person'] = events_df.person.unique()

In [6]:
features_df.head()

,person
0,4886f805
1,ad93850f
2,0297fc1e
3,2d681dd8
4,cccea85e


## Cantidad eventos totales

In [7]:
df = events_df.groupby('person').event.count()

In [8]:
features_df = features_df.join(df, on='person')

In [9]:
features_df.rename(columns={'event':'cant_eventos_totales'}, inplace=True)

In [10]:
features_df.head()

,person,cant_eventos_totales
0,4886f805,9
1,ad93850f,65
2,0297fc1e,567
3,2d681dd8,26
4,cccea85e,836


## Cantidad por evento

In [11]:
df = events_df.groupby('person').event.value_counts()

In [12]:
df = df.unstack()
df.head()

event,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,visited site
person,,,,,,,,,,,
0008ed71,NaN,NaN,3.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.0
00091926,15.0,25.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,372.0,34.0
00091a7a,1.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0
000ba417,1.0,24.0,6.0,1.0,14.0,NaN,1.0,NaN,NaN,153.0,6.0
000c79fe,1.0,NaN,1.0,NaN,1.0,NaN,1.0,9.0,NaN,3.0,1.0


In [13]:
features_df = features_df.join(df, on='person')

In [14]:
features_df.rename(columns={'ad campaign hit':'cant_ach', 'brand listing':'cant_bl', 'checkout':'cant_ch', 
                            'conversion':'cant_cv', 'generic listing':'cant_gl', 'lead':'cant_l',
                           'search engine hit':'cant_seh', 'searched products':'cant_s_p', 
                            'staticpage':'cant_sp', 'viewed product':'cant_vp', 'visited site':'cant_vs'}, inplace=True)

In [15]:
features_df.head()

,person,cant_eventos_totales,cant_ach,cant_bl,cant_ch,cant_cv,cant_gl,cant_l,cant_seh,cant_s_p,cant_sp,cant_vp,cant_vs
0,4886f805,9,NaN,NaN,1.0,NaN,1.0,NaN,1.0,1.0,NaN,4.0,1.0
1,ad93850f,65,10.0,15.0,1.0,NaN,7.0,NaN,7.0,NaN,NaN,20.0,5.0
2,0297fc1e,567,29.0,4.0,7.0,NaN,21.0,1.0,NaN,6.0,NaN,404.0,95.0
3,2d681dd8,26,1.0,5.0,1.0,NaN,1.0,NaN,2.0,1.0,NaN,13.0,2.0
4,cccea85e,836,15.0,7.0,1.0,NaN,20.0,NaN,26.0,1.0,5.0,739.0,22.0


In [16]:
features_df.fillna(0, inplace=True)

In [17]:
features_df.head()

,person,cant_eventos_totales,cant_ach,cant_bl,cant_ch,cant_cv,cant_gl,cant_l,cant_seh,cant_s_p,cant_sp,cant_vp,cant_vs
0,4886f805,9,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,4.0,1.0
1,ad93850f,65,10.0,15.0,1.0,0.0,7.0,0.0,7.0,0.0,0.0,20.0,5.0
2,0297fc1e,567,29.0,4.0,7.0,0.0,21.0,1.0,0.0,6.0,0.0,404.0,95.0
3,2d681dd8,26,1.0,5.0,1.0,0.0,1.0,0.0,2.0,1.0,0.0,13.0,2.0
4,cccea85e,836,15.0,7.0,1.0,0.0,20.0,0.0,26.0,1.0,5.0,739.0,22.0


## Cantidad de returnings

In [18]:
df = events_df.groupby('person').new_vs_returning.value_counts().unstack().drop(columns=['New'])

In [19]:
features_df = features_df.join(df, on='person')

In [20]:
features_df.head()

,person,cant_eventos_totales,cant_ach,cant_bl,cant_ch,cant_cv,cant_gl,cant_l,cant_seh,cant_s_p,cant_sp,cant_vp,cant_vs,Returning
0,4886f805,9,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,4.0,1.0,NaN
1,ad93850f,65,10.0,15.0,1.0,0.0,7.0,0.0,7.0,0.0,0.0,20.0,5.0,4.0
2,0297fc1e,567,29.0,4.0,7.0,0.0,21.0,1.0,0.0,6.0,0.0,404.0,95.0,94.0
3,2d681dd8,26,1.0,5.0,1.0,0.0,1.0,0.0,2.0,1.0,0.0,13.0,2.0,1.0
4,cccea85e,836,15.0,7.0,1.0,0.0,20.0,0.0,26.0,1.0,5.0,739.0,22.0,21.0


## Tiempo total

In [21]:
events_df['timestamp'] = pd.to_datetime(events_df['timestamp'])

In [22]:
df = (events_df.groupby('person').timestamp.agg('max') - events_df.groupby('person').timestamp.agg('min'))
df.head()

person
0008ed71    0 days 04:00:50
00091926   27 days 21:43:34
00091a7a    0 days 00:05:47
000ba417    9 days 01:57:37
000c79fe    0 days 00:10:20
Name: timestamp, dtype: timedelta64[ns]

In [23]:
features_df = features_df.join(df, on='person')

In [24]:
features_df = features_df.rename(columns={'timestamp':'tiempo_total'})

In [25]:
features_df[['person', 'tiempo_total']].head()

,person,tiempo_total
0,4886f805,0 days 00:23:08
1,ad93850f,7 days 23:00:32
2,0297fc1e,138 days 12:53:38
3,2d681dd8,9 days 19:34:20
4,cccea85e,23 days 16:05:31


In [26]:
features_df.tiempo_total = pd.to_numeric(features_df.tiempo_total, downcast='float')
features_df.tiempo_total.head()

0    1.388000e+12
1    6.876320e+14
2    1.196962e+16
3    8.480600e+14
4    2.045131e+15
Name: tiempo_total, dtype: float32

## Tiempo entre new y returning

In [27]:
df_news = events_df.loc[events_df.new_vs_returning == 'New']
df_news[['person', 'timestamp', 'new_vs_returning']].head()

,person,timestamp,new_vs_returning
2136660,f35dddc8,2018-04-11 14:46:36,New
2136661,c82ecb12,2018-04-23 14:56:42,New
2136662,102f402d,2018-05-14 15:56:35,New
2136668,1a6498ed,2018-05-15 18:13:22,New
2136669,f11aace6,2018-05-18 14:44:53,New


In [28]:
df_returnings = events_df.loc[events_df.new_vs_returning == 'Returning']
df_returnings[['person', 'timestamp', 'new_vs_returning']].head()

,person,timestamp,new_vs_returning
2136629,4640420b,2018-05-10 22:34:50,Returning
2136630,4640420b,2018-05-15 02:39:45,Returning
2136631,4640420b,2018-05-18 01:15:26,Returning
2136632,4640420b,2018-05-18 19:03:37,Returning
2136633,4640420b,2018-05-18 19:35:12,Returning


In [29]:
df = (df_returnings.groupby('person').timestamp.min() - df_news.groupby('person').timestamp.min())
df = df.fillna(0)
df.head()

person
0008ed71   0 days 02:36:55
00091926   1 days 05:34:55
00091a7a   0 days 00:00:00
000ba417   0 days 01:45:31
000c79fe   0 days 00:00:00
Name: timestamp, dtype: timedelta64[ns]

In [30]:
features_df = features_df.join(df, on='person')

In [31]:
features_df = features_df.rename(columns={'timestamp':'tiempo_new_returning'})

In [32]:
features_df[['person', 'tiempo_new_returning']].head()

,person,tiempo_new_returning
0,4886f805,0 days 00:00:00
1,ad93850f,1 days 02:57:51
2,0297fc1e,3 days 00:00:24
3,2d681dd8,9 days 19:28:03
4,cccea85e,2 days 21:21:19


In [33]:
features_df.tiempo_new_returning = pd.to_numeric(features_df.tiempo_new_returning, downcast='float')
features_df.tiempo_new_returning.head()

0    0.000000e+00
1    9.707100e+13
2    2.592240e+14
3    8.476830e+14
4    2.496790e+14
Name: tiempo_new_returning, dtype: float32

## Mean entre tiempos de returnings

## Channel más frecuente

In [34]:
def agg_mas_frecuente(x):
    return (x.value_counts().index[0] if(len(x.value_counts())>0) else np.nan)

In [35]:
events_df.groupby('person').channel.value_counts()

person    channel 
0008ed71  Referral      2
00091926  Direct       25
          Paid          8
          Referral      1
00091a7a  Paid          1
000ba417  Direct        6
000c79fe  Paid          1
000e4d9e  Direct        5
          Referral      5
          Paid          2
          Organic       1
000e619d  Direct        2
          Organic       2
          Paid          1
001001be  Organic       3
0010e89a  Paid          1
0016c4b5  Paid          1
001802e4  Paid          1
001804a2  Referral      1
0019e639  Paid         16
          Organic       2
          Referral      1
001a2273  Paid          1
001ca5ee  Referral      6
          Paid          5
          Direct        2
          Organic       2
001dfc31  Paid          3
          Organic       1
001e9aea  Paid          7
                     ... 
ffebdbc9  Paid          1
ffecdf29  Direct        2
          Paid          1
ffecea3d  Organic       1
          Paid          1
ffed0342  Referral     15
          Direct   

In [36]:
df = events_df.groupby('person').channel.agg(lambda x: agg_mas_frecuente(x))
df.head()

person
0008ed71    Referral
00091926      Direct
00091a7a        Paid
000ba417      Direct
000c79fe        Paid
Name: channel, dtype: object

In [37]:
features_df = features_df.join(df, on='person')

In [38]:
features_df.columns

Index(['person', 'cant_eventos_totales', 'cant_ach', 'cant_bl', 'cant_ch',
       'cant_cv', 'cant_gl', 'cant_l', 'cant_seh', 'cant_s_p', 'cant_sp',
       'cant_vp', 'cant_vs', 'Returning', 'tiempo_total',
       'tiempo_new_returning', 'channel'],
      dtype='object')

In [39]:
features_df.rename(columns={'channel':'channel_frecuente'}, inplace=True)

In [40]:
df.value_counts().idxmax()

'Paid'

In [41]:
features_df.channel_frecuente.fillna(df.value_counts().idxmax(), inplace=True)

In [42]:
features_df.channel_frecuente.isna().value_counts()

False    38829
Name: channel_frecuente, dtype: int64

In [43]:
features_df[['person', 'channel_frecuente']].head()

,person,channel_frecuente
0,4886f805,Organic
1,ad93850f,Paid
2,0297fc1e,Direct
3,2d681dd8,Organic
4,cccea85e,Organic


## Device más frecuente

In [44]:
events_df.device_type.value_counts()

Smartphone    103502
Computer       97485
Tablet          2799
Unknown          283
Name: device_type, dtype: int64

In [45]:
df = events_df.groupby('person').device_type.agg(lambda x: agg_mas_frecuente(x))
df.head()

person
0008ed71      Computer
00091926      Computer
00091a7a    Smartphone
000ba417      Computer
000c79fe    Smartphone
Name: device_type, dtype: object

In [46]:
features_df = features_df.join(df, on='person')

In [47]:
features_df.columns

Index(['person', 'cant_eventos_totales', 'cant_ach', 'cant_bl', 'cant_ch',
       'cant_cv', 'cant_gl', 'cant_l', 'cant_seh', 'cant_s_p', 'cant_sp',
       'cant_vp', 'cant_vs', 'Returning', 'tiempo_total',
       'tiempo_new_returning', 'channel_frecuente', 'device_type'],
      dtype='object')

In [48]:
features_df.rename(columns={'device_type':'device_frecuente'}, inplace=True)

In [49]:
df.value_counts().idxmax()

'Smartphone'

In [50]:
features_df.channel_frecuente.fillna(df.value_counts().idxmax(), inplace=True)

In [51]:
features_df[['person', 'device_frecuente']].head()

,person,device_frecuente
0,4886f805,Smartphone
1,ad93850f,Smartphone
2,0297fc1e,Smartphone
3,2d681dd8,Computer
4,cccea85e,Computer


## Quincenas

In [52]:
events_df['timestamp'] = pd.to_datetime(events_df['timestamp'])
events_df['mes'] = events_df['timestamp'].dt.month
events_df['dia'] = events_df['timestamp'].dt.day

In [53]:
events_df['q1'] = ((events_df['mes']==1) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q2'] = ((events_df['mes']==1) & (events_df['dia']>15) & (events_df['dia']<32))
events_df['q3'] = ((events_df['mes']==2) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q4'] = ((events_df['mes']==2) & (events_df['dia']>15) & (events_df['dia']<29))
events_df['q5'] = ((events_df['mes']==3) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q6'] = ((events_df['mes']==3) & (events_df['dia']>15) & (events_df['dia']<32))
events_df['q7'] = ((events_df['mes']==4) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q8'] = ((events_df['mes']==4) & (events_df['dia']>15) & (events_df['dia']<31))
events_df['q9'] = ((events_df['mes']==5) & (events_df['dia']>1) & (events_df['dia']<16))
events_df['q10'] = ((events_df['mes']==5) & (events_df['dia']>15) & (events_df['dia']<32))

In [54]:
df = events_df[['person', 'q1','q2','q3','q4','q5','q6','q7','q8','q9','q10']].groupby('person').agg(lambda x: x.any())
df.head()

,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10
person,,,,,,,,,,
0008ed71,False,False,False,False,False,False,False,False,False,True
00091926,False,False,False,False,False,False,False,False,True,True
00091a7a,False,False,False,False,False,True,False,False,False,False
000ba417,False,False,False,False,False,False,False,False,False,True
000c79fe,False,False,False,False,False,False,False,False,False,True


In [55]:
features_df = features_df.join(df, on='person')

In [56]:
features_df.columns

Index(['person', 'cant_eventos_totales', 'cant_ach', 'cant_bl', 'cant_ch',
       'cant_cv', 'cant_gl', 'cant_l', 'cant_seh', 'cant_s_p', 'cant_sp',
       'cant_vp', 'cant_vs', 'Returning', 'tiempo_total',
       'tiempo_new_returning', 'channel_frecuente', 'device_frecuente', 'q1',
       'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10'],
      dtype='object')

# Predicciones
---

In [57]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

## Separación de datos

In [58]:
labels_df = pd.read_csv("data/fiuba-trocafone-tp2-final-set/labels_training_set.csv", low_memory=False)

In [106]:
train_df = features_df.merge(labels_df, on='person', how='right')
train_df.shape

(19414, 29)

In [107]:
labels = train_df.label
train_df = train_df.drop(columns=['label'])

In [108]:
to_predict = features_df[~features_df.person.isin(labels_df.person)]
to_predict.shape

(19415, 28)

In [109]:
X_train, X_test, y_train, y_test = \
    train_test_split(train_df.drop(columns=['person']) , labels, test_size=0.2, random_state=123)

In [110]:
X_train.reset_index(inplace=True)
X_train.drop(columns='index', inplace=True)

X_test.reset_index(inplace=True)
X_test.drop(columns='index', inplace=True)

y_train = pd.DataFrame(y_train)
y_train.reset_index(inplace=True)
y_train.drop(columns='index', inplace=True)

y_test = pd.DataFrame(y_test)
y_test.reset_index(inplace=True)
y_test.drop(columns='index', inplace=True)

to_predict = pd.DataFrame(to_predict)
to_predict.reset_index(inplace=True)
to_predict.drop(columns='index', inplace=True)

## Encoders

In [111]:
import category_encoders as ce

### Channel frecuente

In [112]:
te_channel_frecuente = ce.TargetEncoder(handle_unknown='ignore')

In [113]:
X_train.channel_frecuente = te_channel_frecuente.fit_transform(pd.DataFrame(X_train.channel_frecuente), y_train.label)

### Device frecuente

In [114]:
te_device_frecuente = ce.TargetEncoder(handle_unknown='ignore')

In [115]:
X_train.device_frecuente = te_device_frecuente.fit_transform(pd.DataFrame(X_train.device_frecuente), y_train.label)

## Modelo

In [118]:
c=lgb.LGBMRegressor(learning_rate=0.5,
                     objective='regression',
                     max_depth=20, 
                     num_leaves=3000,
                     n_estimators=50,
                     colsample_bytree=0.9,
                     n_jobs=1,
                     random_state=0,
                     silent=False,
                     subsample=1,
                     sumsample_freq=1, 
                     two_round=True,
                     boosting_type='dart')

dt=c.fit(X_train,y_train.label,eval_set=[(X_test,y_test.label)],verbose=True,early_stopping_rounds=1)

[1]	valid_0's l2: 0.0392237
Training until validation scores don't improve for 1 rounds.
[2]	valid_0's l2: 0.0394573
Early stopping, best iteration is:
[1]	valid_0's l2: 0.0392237


In [119]:
c.score(X_test,y_test)
#0.9415400463559104

0.0415198483770306

## Tests

In [116]:
X_test.channel_frecuente = te_channel_frecuente.transform(pd.DataFrame(X_test.channel_frecuente))

In [117]:
X_test.device_frecuente = te_device_frecuente.transform(pd.DataFrame(X_test.device_frecuente))

In [ ]:
preds = xg_reg.predict(X_test)

In [ ]:
features_df.columns

In [ ]:
roc_auc_score(y_test,preds)

## Predicciones posta

In [73]:
to_predict.channel_frecuente = te_channel_frecuente.transform(pd.DataFrame(to_predict.drop(columns='person').channel_frecuente))

In [74]:
to_predict.device_frecuente = te_device_frecuente.transform(pd.DataFrame(to_predict.drop(columns='person').device_frecuente))

In [75]:
preds_posta = c.predict_proba(to_predict.drop(columns=['person']))

In [76]:
preds_posta

array([[0.98846273, 0.01153727],
       [0.99408322, 0.00591678],
       [0.97399931, 0.02600069],
       ...,
       [0.89734261, 0.10265739],
       [0.95124281, 0.04875719],
       [0.95124281, 0.04875719]])

In [77]:
preds_posta.shape

(19415, 2)

In [78]:
preds_posta2=[]
print(c.classes_)
print(len(preds_posta))
for elem in preds_posta:
    preds_posta2.append(elem[1])
print(len(preds_posta2))

[0 1]
19415
19415


## Generación del csv a publicar

In [79]:
to_publish = pd.DataFrame()

In [80]:
to_publish['person'] = to_predict.person
to_publish.shape

(19415, 1)

In [81]:
to_publish['label'] = preds_posta2
to_publish.shape

(19415, 2)

In [82]:
to_publish.head()

,person,label
0,4886f805,0.011537
1,0297fc1e,0.005917
2,2d681dd8,0.026001
3,cccea85e,0.013497
4,4c8a8b93,0.010848


In [83]:
to_publish.to_csv('28_11_2.csv', index=False)